In [109]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd

### Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [139]:
def get_filtered_post_data(keywords):
    import requests
    from bs4 import BeautifulSoup
    from datetime import date, timedelta
    import pandas as pd
    res = requests.get('https://habr.com/ru/all')
    soup = BeautifulSoup(res.text, 'html.parser')
    post_time = soup.find_all('span', class_='post__time')
    post_time = [i.text for i in post_time]
    post_title = soup.find_all('h2', class_='post__title')
    post_title_list = list(map(lambda x: x.find('a', class_ = 'post__title_link').text.strip(), post_title))
    post_link_list = list(map(lambda x: x.find('a').get('href'), post_title))
    #преобразуем "сегодня", "вчера" в даты
    today = date.today()
    yesterday = (today - timedelta(days = 1)).strftime("%B %d, %Y")
    today = today.strftime("%B %d, %Y")
    new = []
    for i in post_time:
        if 'сегодня' in i:
            new.append(d2 + ' ' + i[10:])
        elif 'вчера' in i:
            new.append(yesterday + i[7:])
        else:
            new.append(i) 
    # создаем DataFrame
    data = {'Date':  post_time,
        'Title': post_title_list,
         'Link': post_link_list}
    df = pd.DataFrame (data, columns = ['Date','Title', 'Link'])
    return df[df['Title'].str.contains('|'.join(keywords))]

In [140]:
get_filtered_post_data(['сети', 'ядро'])

,Date,Title,Link
1,сегодня в 23:07,Искусственные нейронные сети. Часть 2,https://habr.com/ru/post/542568/
2,сегодня в 22:38,Отлаживаем ядро из командной строки с bpftrace,https://habr.com/ru/post/542560/
15,сегодня в 17:51,Искусственные нейронные сети. Часть 1,https://habr.com/ru/post/542518/


### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [149]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [188]:
email = ['xxx@x.ru', 'yyy@y.com']

In [189]:
payload = {'emailAddresses': email}

In [190]:
headers = {'Accept': 'application/json, text/plain, */*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '40',
'Content-Type': 'application/json;charset=UTF-8',
'Host': 'identityprotection.avast.com',
'Origin': 'https://www.avast.com',
'Referer': 'https://www.avast.com/hackcheck/',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 OPR/73.0.3856.344',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Vaar-Header-Build-Version': '{__VERSION__}',
'Vaar-Version': '0'}

In [191]:
avast_response = requests.post(url, json=payload, headers = headers)

In [192]:
avast_response.status_code

200

In [193]:
avast_response.json()

{'breaches': {'16613': {'breachId': 16613,
   'site': 'verifications.io',
   'recordsCount': 677914246,
   'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n',
   'publishDate': '2019-03-28T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 0, 'emails': 677914246}},
  '17609': {'breachId': 17609,
   'site': 'vk.com',
   'recordsCount': 42536910,
   'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and

In [225]:
breaches = pd.DataFrame(avast_response.json()['breaches']).T

In [226]:
data = pd.DataFrame(avast_response.json()['data']).T

In [227]:
summary = pd.DataFrame(avast_response.json()['summary']).T

In [229]:
breaches.head()

,breachId,site,recordsCount,description,publishDate,statistics
16613,16613,verifications.io,677914246,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z,"{'usernames': 0, 'passwords': 0, 'emails': 677..."
17609,17609,vk.com,42536910,"At some time in 2020, the Russian social netwo...",2020-05-21T00:00:00Z,"{'usernames': 0, 'passwords': 0, 'emails': 205..."
17110,17110,azcentral.com,705538,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z,"{'usernames': 0, 'passwords': 702971, 'emails'..."
17670,17670,wishbone.io,37475476,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z,"{'usernames': 31111127, 'passwords': 10648734,..."
13094,13094,myheritage.com,110041653,"In October 2017, a customer database belonging...",2017-11-04T00:00:00Z,"{'usernames': 0, 'passwords': 110041647, 'emai..."


In [212]:
test = breaches[['site', 'description', 'publishDate']]

In [230]:
data.head()

,xxx@x.ru,yyy@y.com
parapa.mail.ru,"[{'breachId': 3176, 'usernameBreached': True, ...",NaN
verifications.io,"[{'breachId': 16613, 'usernameBreached': True,...",NaN
adobe.com,"[{'breachId': 3, 'usernameBreached': True, 'pa...","[{'breachId': 3, 'usernameBreached': True, 'pa..."
cdprojektred.com,"[{'breachId': 2961, 'usernameBreached': True, ...",NaN
azcentral.com,NaN,"[{'breachId': 17110, 'usernameBreached': True,..."


In [215]:
summary

,breaches
xxx@x.ru,"[16613, 17609, 3176, 12, 3, 3164, 2961, 15]"
yyy@y.com,"[17110, 17670, 13094, 37177, 16768, 41, 2, 358..."


In [246]:
for i,k in summary.iteritems():
    for j in k:
        print(j)

[16613, 17609, 3176, 12, 3, 3164, 2961, 15]
[17110, 17670, 13094, 37177, 16768, 41, 2, 3587, 17009, 3, 13662, 3520, 15, 3669]


In [247]:
test.head()

,site,description,publishDate
16613,verifications.io,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z
17609,vk.com,"At some time in 2020, the Russian social netwo...",2020-05-21T00:00:00Z
17110,azcentral.com,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z
17670,wishbone.io,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z
13094,myheritage.com,"In October 2017, a customer database belonging...",2017-11-04T00:00:00Z


In [250]:
test.iloc[test['16613']]

KeyError: '16613'